# SmartClip AI - Google Colab

Transform long videos into viral short clips using AI.

**Features:** AI-powered clip detection, Auto captions, Multiple aspect ratios, 100% FREE with Groq API

## Step 1: Install SmartClip AI

In [ ]:
# Install SmartClip AI
!pip install -q git+https://github.com/sakirsyarian/sclip.git
!apt-get update -qq && apt-get install -qq -y ffmpeg
!sclip --check-deps
print('Setup complete!')

## Step 2: Configure API Key

Get FREE key from https://console.groq.com

In [ ]:
import os
from getpass import getpass

try:
    from google.colab import userdata
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    if GROQ_API_KEY:
        print('API key loaded from Colab Secrets')
except:
    GROQ_API_KEY = None

if not GROQ_API_KEY:
    GROQ_API_KEY = getpass('Enter Groq API key: ')

os.environ['GROQ_API_KEY'] = GROQ_API_KEY
print('API key configured!')

## Step 3: Upload Video

Choose: Upload (max 100MB), YouTube URL, or Google Drive

In [ ]:
#@title Video Source
video_source = 'YouTube URL' #@param ['Upload from computer', 'YouTube URL', 'Google Drive']
youtube_url = '' #@param {type:'string'}
drive_path = '/content/drive/MyDrive/video.mp4' #@param {type:'string'}

VIDEO_PATH = None
YOUTUBE_URL = None

if video_source == 'Upload from computer':
    from google.colab import files
    print('Select video file (max 100MB)...')
    uploaded = files.upload()
    if uploaded:
        VIDEO_PATH = list(uploaded.keys())[0]
        print(f'Uploaded: {VIDEO_PATH}')
elif video_source == 'YouTube URL':
    if youtube_url:
        YOUTUBE_URL = youtube_url
        print(f'YouTube: {YOUTUBE_URL}')
    else:
        print('Enter YouTube URL above')
elif video_source == 'Google Drive':
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    if os.path.exists(drive_path):
        VIDEO_PATH = drive_path
        print(f'Found: {VIDEO_PATH}')
    else:
        print(f'Not found: {drive_path}')

## Step 4: Generate Clips

In [ ]:
#@title Settings
max_clips = 5 #@param {type:'slider', min:1, max:10, step:1}
aspect_ratio = '9:16' #@param ['9:16', '1:1', '16:9']
caption_style = 'default' #@param ['default', 'bold', 'minimal', 'karaoke']
language = 'id' #@param ['id', 'en', 'auto']
dry_run = False #@param {type:'boolean'}

cmd = 'sclip'
if YOUTUBE_URL:
    cmd += f' -u "{YOUTUBE_URL}"'
elif VIDEO_PATH:
    cmd += f' -i "{VIDEO_PATH}"'
else:
    raise Exception('No video! Run Step 3 first.')

cmd += f' -o /content/output -n {max_clips} -a {aspect_ratio} -s {caption_style} -l {language}'
if dry_run:
    cmd += ' --dry-run'

print(f'Running: {cmd}')
!{cmd}

## Step 5: Download Clips

In [ ]:
#@title Download
download_method = 'Download to computer' #@param ['Download to computer', 'Save to Google Drive']
drive_folder = '/content/drive/MyDrive/SmartClip_Output' #@param {type:'string'}

import glob, os, shutil
clips = glob.glob('/content/output/*.mp4')

if not clips:
    print('No clips found. Run Step 4 first.')
else:
    print(f'Found {len(clips)} clip(s)')
    if download_method == 'Download to computer':
        from google.colab import files
        for f in clips:
            files.download(f)
        print('Download complete!')
    else:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        os.makedirs(drive_folder, exist_ok=True)
        for f in clips:
            shutil.copy(f, drive_folder)
        print(f'Saved to {drive_folder}')

---

## Tips

- Enable GPU: Runtime > Change runtime type > T4 GPU
- Save API key in Colab Secrets (key icon in sidebar)
- For large videos (>100MB), use YouTube URL or Google Drive
- Use dry_run=True to preview before rendering